**<font color = blue size=4>第三部分:实验任务 Part II--构建决策树</font>**

<span style="color:purple">用【ID3】算法实现一棵完整的决策树。完成DTree类中的TreeGenerate、train函数以完成决策树的构建。并完成DTree类中的predict函数来用构建好的决策树来对测试数据集进行预测并输出预测准确率。</span>

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from collections import Counter

定义计算熵的公式，定义划分不同数据集的函数，定义信息增益最大的最优划分

In [33]:
#定义计算信息熵函数
def entropy(label):
    counter = Counter(label)
    ent = 0.0
    for i in counter.values():
        ent = ent - i / len(label) * math.log2(i / len(label))
    return ent
#定义划分不同数据集函数
def split(feature, label, dimension):
    unFeature = np.unique(feature[:,dimension])
    unFeatureNum = len(unFeature)
    
    split_feature = list()
    split_label = list()
    splitElementF = list()
    splitElementL = list()
    
    for i in range(unFeatureNum):
        splitElementF.append([])
        splitElementL.append([])
    
    for i in range(len(label)):
        for j in range(unFeatureNum):
            if feature[i, dimension] == unFeature[j]:
                splitElementF[j].append(feature[i,:])
                splitElementL[j].append(label[i])
                break;
    
    for i in range(unFeatureNum):
        split_feature.append(np.array(splitElementF[i]))
        split_label.append(np.array(splitElementL[i]))
    
    return split_feature,split_label

#定义信息增益最大的最优划分
def one_split_ID3(feature, label):
    
    gain = list()
    entD = entropy(label)
    
    for i in range(feature.shape[1]):
        split_feature,split_label = split(feature, label, i)
        s = 0.0
        for j in range(len(split_label)):
            entDv = entropy(split_label[j])
            s = s + entDv * len(split_label[j]) / feature.shape[0]
        gain.append(entD - s)
    
    best_entropy = -1.0
    best_dimension = -1
    
    for i in range(feature.shape[1]):
        if gain[i] > best_entropy:
            best_entropy = gain[i]
            best_dimension = i
            
    return best_entropy, best_dimension


def best_split(D, A):
    
    label = D[:,-1]
    feature = []
    for i in range(D.shape[1]-1):
        if A[i] == 1:
            feature.append(D[:,i])
    f = np.array(feature)
    best_entropy, best_dimension = one_split_ID3(f.T, label)
    b = best_dimension
    for i in range(D.shape[1]-1):
        if A[i] == 1:
            b = b - 1
            if b == -1:
                best_dimension = i
    
    return best_dimension

建立树的结点

In [34]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.index = index # index表示该分支结点的划分属性的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分属性的值为val的孩子结点

决策树类

In [35]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None #决策树的根结点
        self.possible_value = {} # 用于存储每个属性可能的取值
    def TreeGenerate(self, D, A):
        
        # 生成结点 node
        node = Node()
        label = D[:,-1]
        label_unique = np.unique(label)
        
        if len(label_unique) == 1:
            node.isLeaf = True
            node.label = label_unique[0]
            return node
        t1 = 0
        t2 = 0
        for i in range(len(A)):
            if A[i] == 1:
                t1 = t1 + 1
                feature_unique = np.unique(D[:,i])
                if len(feature_unique) != 1:
                    t2 = t2 + 1
        if t1 == 0 or t2 == 0:
            node.isLeaf = True
            counter = Counter(label)
            counter_values = list(counter.values())
            counter_label = list(counter)
            max = counter_values[0]
            node.label = counter_label[0]
            for i in range(len(counter_values)):
                if counter_values[i] > max:
                    max = counter_values[i]
                    node.label = counter_label[i]
            return node
        best_dimension = best_split(D, A)
        feature_a_unique = self.possible_value[best_dimension]
        
        for i in range(len(feature_a_unique)):
            newNode = Node()
            node.index = best_dimension
            node.isLeaf = False
            Dv = []
            for j in range(D.shape[0]):
                if D[j, best_dimension] == feature_a_unique[i]:
                    Dv.append(D[j])
            D_v = np.array(Dv)
            if len(D_v) == 0:
                newNode.isLeaf = True
                counter = Counter(label)
                counter_values = list(counter.values())
                counter_label = list(counter)
                max = counter_values[0]
                newNode.label = counter_label[0]
                for i in range(len(counter_values)):
                    if counter_values[i] > max:
                        max = counter_values[i]
                        newNode.label = counter_label[i]
                node.addNode(feature_a_unique[i], newNode)
            else:
                copyA = A[:]
                copyA[best_dimension] = 0
                node.addNode(feature_a_unique[i], self.TreeGenerate(D_v, copyA))
        
        
        return node
    def train(self, D):
        D = np.array(D)
        A = []
        for i in range(D.shape[1]-1):
            A.append(1)
        for i in range(len(A)):
            self.possible_value[i] = np.unique(D[:,i])
        self.tree_root = self.TreeGenerate(D, A)
        
        pass
    def predict(self, D):
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        
#         #对于D中的每一行数据d，从当前结点x=self.tree_root开始，当当前结点x为分支结点时，
#         #则搜索x的划分属性为该行数据相应的属性值的孩子结点（即x=x.children[d[x.index]]），不断重复，
#         #直至搜索到叶结点，该叶结点的label就是数据d的预测label
        D = np.array(D)
        pre = []
        count = 0
        for i in range(D.shape[0]):
            d = D[i]
            x = self.tree_root
            while x.isLeaf == False:
                x = x.children[d[x.index]]
            pre.append(x.label)
            if x.label == d[-1]:
                count = count + 1
        accuracy = count/D.shape[0]
        print("accuracy:", accuracy)
        pass

预测并输出结果

In [36]:
train_frame = pd.read_csv('train_titanic.csv')
dt = DTree()

# 构建决策树
dt.train(train_frame)

# 利用构建好的决策树对测试数据集进行预测，输出预测准确率（预测正确的个数 / 总数据数量）
test_frame = pd.read_csv('test_titanic.csv')
dt.predict(test_frame)

accuracy: 0.8316831683168316
